파이토치로 배치 학습을 실습합니다.

In [1]:
# 실습에 필요한 라이브러리를 불러옵니다.
from sklearn import datasets
from torch.utils.data import TensorDataset, DataLoader
import torch
import torch.nn as nn
import torch.optim as optim

In [2]:
# 사이킷런에서 유방암 진단 데이터 세트를 불러옵니다.
dataset = datasets.load_breast_cancer()

# 입력 데이터와 타깃을 준비합니다.
X, y = dataset['data'], dataset['target']

# 입력 데이터와 타깃을 텐서 자료구조로 변환합니다.
X = torch.FloatTensor(X)
y = torch.FloatTensor(y).view(-1, 1)

# 데이터 범위 조정을 위해 표준화를 적용합니다.
X = (X - torch.mean(X)) / torch.std(X)

In [3]:
# 입력 데이터와 타깃을 묶어 텐서 데이터 세트를 생성합니다.
dset = TensorDataset(X, y)

# 한 번에 256개의 데이터 샘플을 배치로 사용하는 데이터 로더를 생성합니다.
loader = DataLoader(dset, batch_size=256, shuffle=True)

In [4]:
# n개의 값을 입력 받는 신경망 모델 클래스를 정의합니다.
class NeuralNetwork(nn.Module):
    # 생성자에서 모델의 구조를 정의합니다. 
    def __init__(self, num_features):
        # 상속받아 생성한 객체이므로 부모(nn.Module)의 생성자를 호출합니다.
        super().__init__()
        # num_features개의 특성을 입력받는 은닉층 노드를 4개 생성합니다.
        self.linear1 = nn.Linear(num_features, 4)
        # 렐루 함수 객체를 생성합니다.
        self.relu = nn.ReLU()
        # 4개의 값을 입력받는 출력층 노드 1개를 생성합니다.
        self.linear2 = nn.Linear(4, 1)
        # 시그모이드 함수 객체를 생성합니다.
        self.sigmoid = nn.Sigmoid()

    # 모델의 순전파를 정의합니다.
    def forward(self, X):
        # 생성자에서 만든 은닉층과 출력층 노드로 타깃을 예측하고 반환합니다.
        out = self.linear1(X)
        out = self.relu(out)
        out = self.linear2(out)
        out = self.sigmoid(out)
        return out

In [11]:
# 30개의 값을 입력 받는 로지스틱 회귀 모델 객체를 생성합니다.
model = NeuralNetwork(30)

# 이진 크로스 엔트로피(Binary Cross Entropy) 손실 함수 객체를 생성합니다.
criterion = nn.BCELoss()

# 확률적 경사 하강법 옵티마이저 객체를 생성합니다.
optimizer = optim.SGD(model.parameters(), lr=0.1)

In [12]:
# 학습 함수를 정의합니다.
def train(model, criterion, optimizer, loader):
  # 현재 에포크의 오차를 저장할 변수를 생성합니다.
  epoch_loss = 0

  # 배치 학습을 실행합니다.
  for X_batch, y_batch in loader:    
    # 기울기를 초기화합니다.
    optimizer.zero_grad()
    # 모델을 사용해 타깃을 예측합니다.
    hypothesis = model(X_batch)
    # 손실 함수로 오차를 계산합니다.
    loss = criterion(hypothesis, y_batch)        
    # 기울기를 계산합니다.
    loss.backward()
    # 경사 하강법으로 가중치를 수정합니다.
    optimizer.step()
    # 현재 배치의 오차를 기록합니다.
    epoch_loss += loss.item()

  # 현재 에포크의 오차를 반환합니다.
  return epoch_loss / len(loader)

In [14]:
# 100회에 걸쳐 모델을 학습시킵니다.
n_epochs = 100
for epoch in range(1, n_epochs+1):
  # 모델을 학습시킵니다.
  loss = train(model, criterion, optimizer, loader)  
  # 10 epoch마다 오차를 출력합니다.
  if epoch % 10 == 0:
    print('epoch: {}, loss: {:.4f}'.format(epoch, loss))

epoch: 10, loss: 0.5619
epoch: 20, loss: 0.4354
epoch: 30, loss: 0.3598
epoch: 40, loss: 0.3158
epoch: 50, loss: 0.2917
epoch: 60, loss: 0.2645
epoch: 70, loss: 0.2433
epoch: 80, loss: 0.2487
epoch: 90, loss: 0.2306
epoch: 100, loss: 0.2267


In [15]:
# 학습된 모델을 사용해 결과를 예측합니다.
y_predicted = (model(X) >= 0.5).float()

# 정확도를 계산하고 출력합니다.
score = (y_predicted==y).float().mean()
print('accuracy: {:.2f}'.format(score))

accuracy: 0.92
